In [2]:
from cloudvolume import CloudVolume
import numpy as np
from copy import deepcopy

In [3]:
z1_cv = CloudVolume("gs://dacey-human-retina-001-alignment-temp/rigid_transforms_3x3_LMedS/z-1", fill_missing=True)
z2_cv = CloudVolume("gs://dacey-human-retina-001-alignment-temp/rigid_transforms_3x3_LMedS/z-2", fill_missing=True)
# z1_rev_cv = CloudVolume("gs://dacey-human-retina-001-alignment-temp/rigid_transforms_3x3_LMedS/z1", fill_missing=True)
# z2_rev_cv = CloudVolume("gs://dacey-human-retina-001-alignment-temp/rigid_transforms_3x3_LMedS/z2", fill_missing=True)

z1 = z1_cv[:,:,:]
z2 = z2_cv[:,:,:]
# z1_rev = z1_rev_cv[:,:,:]
# z2_rev = z2_rev_cv[:,:,:]

Downloading: 100%|██████████| 3029/3029 [00:24<00:00, 123.09it/s]


In [185]:
###################################
# Combine rigid transformations estimates from z-1 and z-2 to get
# uninterrupted chain of relative rigid transformations
###################################

IDENTITY = np.eye(2, 3, order='F').reshape(2,3,1,1)

# Use z-1 when existing, otherwise use z-2
rigid_rel = np.where(np.sum(z1, axis=(0,1)) == 0, z2, z1)

# Remaining gaps (likely empty section) are filled with identity
rigid_rel[:,:,np.where(np.sum(rigid_rel, axis=(0,1)) == 0)[0]] = IDENTITY

# Also need to expand the 2x3 rigid transformation to 3x3
new_row = np.array([0, 0, 1]).reshape(1, 3, 1, 1)
new_row_repeated = np.tile(new_row, (1, 1, rigid_rel.shape[2], 1))
rigid_rel = np.vstack((rigid_rel, new_row_repeated), dtype=np.float64)

# Start relative determined by calculating absolute transformations first
# and picking section close to zero rotation (z=2165).
# Translation is just to push everything a bit more into the center
rigid_rel[:,:,0,0] = np.array(
    [[ 0.99373507,  0.11181485, 192./512.  ],
     [-0.11181485,  0.99373507, 32./512.  ],
     [ 0.        ,  0.        , 1.        ]]
)

In [186]:
#####################################
# Compute absolute rigid transformations
#####################################

# Compute the absolute transformations
absolute_transforms_fwd = [rigid_rel[:, :, 0].squeeze()]
for z in range(1, rigid_rel.shape[2]):
    absolute_transform = np.matmul(absolute_transforms_fwd[-1], rigid_rel[:, :, z].squeeze())
    absolute_transforms_fwd.append(absolute_transform)

for z in range(len(absolute_transforms_fwd)):
    absolute_transforms_fwd[z] = np.linalg.inv(absolute_transforms_fwd[z])



In [174]:
absolute_transforms_fwd[0], absolute_transforms_fwd[1], absolute_transforms_fwd[2]

(array([[ 0.99372319, -0.11181351, -0.22047742],
        [ 0.11181351,  0.99372319, -0.27638418],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.99671363, -0.080932  , -0.19746168],
        [ 0.080932  ,  0.99671363, -0.28212666],
        [ 0.        ,  0.        ,  1.        ]]),
 array([[ 0.99671157, -0.08095737, -0.19684708],
        [ 0.08095737,  0.99671157, -0.28271061],
        [ 0.        ,  0.        ,  1.        ]]))

In [187]:
absolute_cv = CloudVolume("gs://dacey-human-retina-001-alignment-temp/rigid_transforms_3x3_LMedS/absolute", delete_black_uploads=True, info=deepcopy(z1_cv.info), fill_missing=True)
absolute_cv.commit_info()
absolute_cv[:,:,1:len(absolute_transforms_fwd)+1,0] = np.array(absolute_transforms_fwd)[:, :2, :3].transpose(1, 2, 0).astype(np.float32)


Uploading: 100%|██████████| 3029/3029 [00:33<00:00, 91.54it/s] 


In [179]:
np.array(absolute_transforms_fwd)[:, :2, :3]

array([[[ 0.99372319, -0.11181351, -0.22047742],
        [ 0.11181351,  0.99372319, -0.27638418]],

       [[ 0.99671363, -0.080932  , -0.19746168],
        [ 0.080932  ,  0.99671363, -0.28212666]],

       [[ 0.99671157, -0.08095737, -0.19684708],
        [ 0.08095737,  0.99671157, -0.28271061]],

       ...,

       [[ 0.9987913 ,  0.04921285, -0.01474322],
        [-0.04921285,  0.9987913 , -0.38784194]],

       [[ 0.99879928,  0.04905047, -0.01471823],
        [-0.04905047,  0.99879928, -0.38782827]],

       [[ 0.99878801,  0.04928012, -0.01681296],
        [-0.04928012,  0.99878801, -0.38868754]]])